In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os, time, shutil, sys
from pathlib import Path
from glob import glob
import numpy as np
import spikeinterface.full as si
import h5py
import matplotlib.pyplot as plt
from tqdm import tqdm
from pprint import pprint

sys.path.append("/home/phornauer/Git/axon_tracking/")
from axon_tracking import spike_sorting as ss

In [ ]:
sorter = 'kilosort2_5'
si.Kilosort2_5Sorter.set_kilosort2_5_path('/home/phornauer/Git/Kilosort_2020b')
sorter_params = si.get_default_sorter_params(si.Kilosort2_5Sorter)
sorter_params['n_jobs'] = -1
sorter_params['detect_threshold'] = 6
sorter_params['minFR'] = 0.01
sorter_params['minfr_goodchannels'] = 0.01
sorter_params['keep_good_only'] = False
sorter_params['do_correction'] = False
sorter_params['NT'] = 64*1024 + 64

Option 1: We only want to sort individual recordings, then we can just insert the input path

In [ ]:
path = '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/lododi/240712/M05180/Network/000178/data.raw.h5'
h5 = h5py.File(path)
stream_ids = list(h5['wells'].keys())
pprint(stream_ids)

Optionally we can also remove specific wells that we do not want to have sorted

In [ ]:
stream_ids.pop(2)
pprint(stream_ids)

In [ ]:
save_root = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Lorenzo/Test'
for stream_id in tqdm(stream_ids):
    rec = si.MaxwellRecordingExtractor(path, stream_id=stream_id)
    ss.clean_sorting(rec, save_root, stream_id=stream_id, sorter=sorter, sorter_params=sorter_params)

Option 2: We saved recordings following a specific pattern and want to sort all of them

In [10]:
root_path = "/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/cmartins/Recordings/ST2036" # Fixed path root that all recordings have in common
path_pattern = ["ST*", "24*","T0027*","Network","0*"] # Variable part of the path, where we collect all possible combinations using wildcards (*). It is still recommended to be as specific as possible to avoid ambiguities.
file_name = "data.raw.h5" # File name of the recording

full_path = os.path.join(root_path, *path_pattern, file_name)
path_list = glob(full_path)
print(f'Found {len(path_list)} recording paths matching the description:\n{full_path}\n')

Found 17 recording paths matching the description:
/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/cmartins/Recordings/ST2036/ST*/24*/T0027*/Network/0*/data.raw.h5



Then we need to specify a logic for how to transform the recording path into the save path for the sorting:

In [8]:
save_path_changes = {'pos': [0, 8, 9, 14, 15, 16], 'vals': ['/', 'phornauer','NEST','','','']}

save_path = ss.convert_rec_path_to_save_path(full_path, save_path_changes)
    
print(f'The save path corresponds to the pattern:\n {save_path}\n')

The save path corresponds to the pattern:
 /net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/NEST/ST2036/ST*/240621/T0027*/



The sorting will automatically create folders for each well of a recording file.

Optionally, we can also here specify recordings that we do not want sorted.

In [11]:
path_list.sort()
path_list.pop(2)
pprint(path_list)

['/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/cmartins/Recordings/ST2036/ST2036AC/240430/T002735/Network/000015/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/cmartins/Recordings/ST2036/ST2036AC/240507/T002735/Network/000007/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/cmartins/Recordings/ST2036/ST2036AC/240528/T002735/Network/000012/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/cmartins/Recordings/ST2036/ST2036AC/240611/T002735/Network/000008/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/cmartins/Recordings/ST2036/ST2036AC/240628/T002735/Network/000033/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/cmartins/Recordings/ST2036/ST2036AC/240705/T002735/Network/000033/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/cmartins/Recordings/ST2036/ST2036AC/2

Then we can do the actual sorting:

In [ ]:
for path in path_list:
    save_root = ss.convert_rec_path_to_save_path(path, save_path_changes)
    h5 = h5py.File(path)
    stream_ids = list(h5['wells'].keys())
    for stream_id in tqdm(stream_ids):
        rec = si.MaxwellRecordingExtractor(path, stream_id=stream_id)
        ss.clean_sorting(rec, save_root, stream_id=stream_id, sorter=sorter, sorter_params=sorter_params)